In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import time
import datetime
from sklearn.metrics import mean_absolute_error, mean_squared_error
# Parameter windowing
input_width = 60*2
label_width = 60
shift = label_width
total_window_size = input_width + shift
OUT_STEPS = label_width
# Definisikan irisan untuk input dan label
input_slice = slice(0, input_width)
label_start = total_window_size - label_width
labels_slice = slice(label_start, None)
train_df = None
output_selected=['RRU.PrbUsedDl']
train_name_cells=['S1/B2/C1']
test_name_cells=['S7/B2/C1']
file_path="D:\\KULIAH\\teep\\AI\\dataset\\08_01_2024\\CellReports.csv"

In [2]:
def periodic_coding(timestamps, max_values):
    sin_features = np.sin(2 * np.pi * timestamps / max_values)
    cos_features = np.cos(2 * np.pi * timestamps / max_values)
    periodic_features = np.concatenate([sin_features, cos_features], axis=-1)
    return periodic_features

def  preprocess_data(file_path):
    df_1=pd.read_csv(file_path)
    convert_time=pd.to_datetime(df_1['timestamp'], unit='ms',origin='unix')
    df_1.insert(df_1.columns.get_loc('timestamp') + 1, 'datetime_column', convert_time)
    df_1.insert(df_1.columns.get_loc('datetime_column') + 2, 'hour', df_1['datetime_column'].dt.hour+df_1['datetime_column'].dt.minute/60)
    df_1.set_index('datetime_column', inplace=True)
    df_1.drop(columns=['timestamp'], inplace=True)
    df_1['sin_time'] = np.sin(df_1['hour'] * (2 * np.pi / 24))
    df_1['cos_time'] = np.cos(df_1['hour']* (2 * np.pi / 24))
    seleted_columns = ['Viavi.Cell.Name','RRU.PrbUsedDl', 'sin_time', 'cos_time'] 
    df_2= df_1[seleted_columns].copy()
    cell_name= train_name_cells+test_name_cells
    df= df_2[df_2['Viavi.Cell.Name'].isin(cell_name)].copy()
    
    return df, cell_name

def make_windows(data_x,data_y, total_window_size, input_slice, labels_slice):
    x = []
    y = []
    for i in range(len(data_x) - total_window_size + 1):
        window_x = data_x[i:i+total_window_size]
        x.append(window_x[input_slice])

    for i in range(len(data_y) - total_window_size + 1):
        window_y= data_y[i:i+total_window_size]
        y.append(window_y[labels_slice])

    return np.array(x), np.array(y)

def standardize_data(data, train_df, isoutput=True, column_output=output_selected):
    if isoutput:
        median = train_df[column_output].median().values
        q1 = train_df[column_output].quantile(0.25).values
        q3 = train_df[column_output].quantile(0.75).values
    else:
        median = train_df.median().values
        q1 = train_df.quantile(0.25).values
        q3 = train_df.quantile(0.75).values

    iqr = q3 - q1

    # Reshape for broadcasting with 2D matrix
    median = median.reshape(1, -1)
    iqr = iqr.reshape(1, -1)

    return (data - median) / iqr

def inverse_standardize_data(data, train_df, isoutput=True, column_output=output_selected):
    if isoutput:
        median = train_df[column_output].median().values
        q1 = train_df[column_output].quantile(0.25).values
        q3 = train_df[column_output].quantile(0.75).values
    else:
        median = train_df.median().values
        q1 = train_df.quantile(0.25).values
        q3 = train_df.quantile(0.75).values

    iqr = q3 - q1

    # Reshape for broadcasting with 2D matrix
    median = median.reshape(1, -1)
    iqr = iqr.reshape(1, -1)

    return data * iqr + median

In [3]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import time
import tensorflow_addons as tfa
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='tensorflow_addons')

# Define the PinballLoss function
pinball_loss = tfa.losses.PinballLoss(tau=0.5, reduction=tf.keras.losses.Reduction.AUTO, name='pinball_loss')

def tensorflow_cnn(X_train_scaled, Y_train_scaled, X_validation_scaled, Y_validation_scaled, 
                    learning_rate, target_error, max_epochs, max_sampel_batch,
                    patience, save_best_model_path, validation_data=False, load_model=None, out_steps=OUT_STEPS):
    global model

    class MAEStopCallback(tf.keras.callbacks.Callback):
        def __init__(self, threshold):
            super(MAEStopCallback, self).__init__()
            self.threshold = threshold

        def on_epoch_end(self, epoch, logs=None):
            if logs['mae'] < self.threshold:
                print(f"\nMAE reached below {self.threshold}. Stopping training.")
                self.model.stop_training = True

                
    input_width = X_train_scaled.shape[1]
    CONV_WIDTH = input_width # Define the width of the convolutional window
    num_features = X_train_scaled.shape[2]
    num_output = Y_train_scaled.shape[2]
    out_steps = out_steps

    if load_model is None:
        print("Create new model")

        # Define the model
        inputs = tf.keras.Input(shape=(input_width, num_features))

        # Initial Conv1D layer
        layer_first = tf.keras.layers.Conv1D(filters=64, kernel_size=8, dilation_rate=1, 
                                            activation='relu', padding='same')(inputs)

        # Residual connection initialization
        residual_before = tf.keras.layers.Conv1D(filters=24, kernel_size=8, dilation_rate=1, activation='relu', padding='same')(layer_first)

        # Residual block with multiple dilations
        for _ in range(8):
            for dilation in (1, 2, 4):
                residual = tf.keras.layers.Conv1D(filters=24, kernel_size=8, dilation_rate=dilation, activation='relu', padding='same')(residual_before)
                residual = tf.keras.layers.BatchNormalization()(residual)
                residual = tf.keras.layers.Dropout(0.05)(residual)
                residual = tf.keras.layers.Add()([residual_before, residual])
                residual_before = residual

        # Output layer
        last_cnn= tf.keras.layers.Conv1D(filters=60, kernel_size=input_width, dilation_rate=1, activation='linear', name='last_cnn')(residual_before)
        output = tf.keras.layers.Dense(out_steps * num_output, name='output_layer')(last_cnn)
        output=tf.keras.layers.Reshape([out_steps, num_output])(output)
        # Define the model
        model = tf.keras.Model(inputs=inputs, outputs=output)
        model.summary()
    else:
        print("Load model")
        model = tf.keras.models.load_model(load_model)

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate,amsgrad=True)
    model.compile(optimizer=optimizer, 
              loss=pinball_loss, 
              metrics=['mse', 'mae', 'mape', tf.keras.metrics.RootMeanSquaredError(name='rmse')])
    mae_stop_callback = MAEStopCallback(threshold=target_error)
    checkpoint_callback = ModelCheckpoint(
        save_best_model_path,
        monitor='val_loss',     
        mode='min',         
        save_best_only=True, 
        verbose=1            
    )

    early_stopping_callback = EarlyStopping(
        monitor='val_loss',     
        mode='min',         
        patience=patience,    
        restore_best_weights=True,
        verbose=1            
    )

    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.3, patience=int(5), min_lr=0.00001, verbose=1)

    time_start = time.time()
    if validation_data:
        model.fit(X_train_scaled, Y_train_scaled, epochs=max_epochs, batch_size=max_sampel_batch,  
                  callbacks=[mae_stop_callback, checkpoint_callback, early_stopping_callback, reduce_lr], 
                  validation_data=(X_validation_scaled, Y_validation_scaled), validation_batch_size=max_sampel_batch)
    else:
        model.fit(X_train_scaled, Y_train_scaled, epochs=max_epochs, batch_size=max_sampel_batch, 
                  callbacks=[mae_stop_callback, checkpoint_callback, early_stopping_callback, reduce_lr])
    
    print("time computation seconds: ", time.time() - time_start)
    
    loss, MSE, MAE, RMSE, MAPE = model.evaluate(X_train_scaled, Y_train_scaled)
    print("loss: ", loss, "MSE: ", MSE, "MAE: ", MAE, "RMSE: ", RMSE, "MAPE: ", MAPE)
    
    return model, loss, MSE, MAE, RMSE, MAPE


d:\GAME\Anaconda\envs\tf_gpu_env\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
d:\GAME\Anaconda\envs\tf_gpu_env\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.11.0 and strictly below 2.14.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want

In [4]:
def running_program(train_df,val_df, index_cell, name_file, name_file_before):
    train_scaled = standardize_data(train_df, isoutput=False, train_df=train_df)
    val_scaled= standardize_data(val_df, isoutput=False, train_df=train_df)
    # Membuat windowed dataset untuk set pelatihan, validasi, dan pengujian

    x_train_scaled, y_train_scaled = make_windows(train_scaled.to_numpy(), train_scaled[output_selected].to_numpy(),total_window_size, input_slice, labels_slice)
    x_val_scaled, y_val_scaled = make_windows(val_scaled.to_numpy(), val_scaled[output_selected].to_numpy(),total_window_size, input_slice, labels_slice)
    model, loss, MSE, MAE, RMSE,  MAPE  = tensorflow_cnn(x_train_scaled, y_train_scaled, x_val_scaled, y_val_scaled,
                                                        learning_rate=0.01, target_error=0.001,  max_epochs=100, max_sampel_batch=128, 
                                                        patience=30,  save_best_model_path = name_file, 
                                                        validation_data=True, load_model=None, out_steps=OUT_STEPS)
    model.save("TCSM_CNN.hdf5")

In [5]:
df_start, cell_name= preprocess_data("D:\\KULIAH\\teep\\AI\\dataset\\08_01_2024\\CellReports.csv")
#now = datetime.datetime.now()
timestamp = "cekpoint"
for index in range(0,1):
    print(index)

    train_df = df_start[df_start['Viavi.Cell.Name'] == train_name_cells[index]]
    train_df = train_df.loc[~train_df.index.duplicated()]
    train_df=train_df.drop(columns=['Viavi.Cell.Name']).astype(float).copy()

    val_df= df_start[df_start['Viavi.Cell.Name'] == test_name_cells[0]]
    val_df=val_df.loc[~val_df.index.duplicated()]
    val_df=val_df.drop(columns=['Viavi.Cell.Name']).astype(float).copy()

    name_file='2hour_%s_%s.hdf5'%(timestamp, index+1)
    name_file_before='2hour_%s_%s.hdf5'%(timestamp, index)
    print("name_file: ", name_file)
    print("name_file_before: ", name_file_before)
    print("Cell Name: ", cell_name[index])
    display(train_df)
    running_program(train_df=train_df, val_df=val_df, index_cell=index, name_file=name_file, name_file_before=name_file_before)

0
name_file:  2hour_cekpoint_1.hdf5
name_file_before:  2hour_cekpoint_0.hdf5
Cell Name:  S1/B2/C1


,RRU.PrbUsedDl,sin_time,cos_time
datetime_column,,,
2024-08-01 00:00:00,35.200000,0.000000,1.000000
2024-08-01 00:01:00,20.116667,0.004363,0.999990
2024-08-01 00:02:00,30.800000,0.008727,0.999962
2024-08-01 00:03:00,34.450000,0.013090,0.999914
2024-08-01 00:04:00,25.900000,0.017452,0.999848
...,...,...,...
2024-08-08 23:55:00,25.116667,-0.021815,0.999762
2024-08-08 23:56:00,24.666667,-0.017452,0.999848
2024-08-08 23:57:00,59.133333,-0.013090,0.999914


Create new model
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 120, 3)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 120, 64)      1600        ['input_1[0][0]']                
                                                                                                  
 conv1d_1 (Conv1D)              (None, 120, 24)      12312       ['conv1d[0][0]']                 
                                                                                                  
 conv1d_2 (Conv1D)              (None, 120, 24)      4632        ['conv1d_1[0][0]']               
                                                                             